In [1]:
# Importing some shizzle.

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import time
import requests
import datetime
import sys
import os
import pathlib
import seaborn as sns
from sklearn.metrics import roc_auc_score
import math
import json
import glob

# Defining some import and export locations
location = 'rjsietsma'
read_loc = '/home/'+location+'/Documents/School/Master_DSLS/Final_Thesis/Initial_Data_exploration/'
data_expor_loc = '/home/'+location+'/Documents/School/Master_DSLS/Final_Thesis/Past_initial_data/'
img_output_dir = '/home/'+location+'/PycharmProjects/dsls_master_thesis/side_scripts/output_img/'

In [2]:
# Some bokeh stuff

from bokeh.plotting import figure, ColumnDataSource, output_file, show, save
from bokeh.models import HoverTool, WheelZoomTool, PanTool, BoxZoomTool, ResetTool, TapTool, SaveTool, FactorRange
from bokeh.palettes import inferno, Spectral5
from bokeh.transform import factor_cmap

In [3]:
# Define function to perform Shapiro-Wilk, Kolmogorov-Smirnov, Wilcoxon-/Mann-Whitney U -test, and a pearson correlation test.

def perform_stats(x, y):
    shapiro_x = stats.shapiro(x)[1]
    shapiro_y = stats.shapiro(y)[1]
    if shapiro_x >= 0.05:
        print("X is likely normally distributed.")
    else:
        print("X is likely not normally distributed.")
    if shapiro_y >= 0.05:
        print("Y is likely normally distributed.")
    else:
        print("Y is likely not normally distributed.")
    try:
        p_ks_xy = stats.ks_2samp(x, y)[1]
        print(f"The Kolmogorov-Smirnov test p-value: {p_ks_xy}")
    except Exception:
        print("Kolmogorov-Smirnov could not be performed!")
    try:
        p_wc_xy = stats.wilcoxon(x, y)[1]
        print(f"The Wilcoxon test p-value: {p_wc_xy}")
    except ValueError:
        p_mw_xy = stats.mannwhitneyu(x, y)[1]
        print(f"Wilcoxon could not be performed, \n"
             f"Using Mann-Whitney rank test p-value: {p_mw_xy}")
    except Exception:
        print("Neither Wilcoxon nor Mann-Whitney tests could be performed!")
    try:
        p_pears_xy = stats.pearsonr(x, y)
        print(f"The Pearson correlation: {p_pears_xy[0]},\n"
             f"p-value: {p_pears_xy[1]}")
    except Exception:
        print("Pearson correlation could not be performed!")

# Define function to calculate the Z-scores of given data.

def calc_z_scores(data):
    centered = data - data.mean(axis=0)
    return centered / centered.std(axis=0)

Sprint 28-04-2020 - 19-05-2020 (?)
=====================

- What I want to do:
  1. [Focus on fixing missing CAPICE precomputed scores.](#Fixing-CAPICE-precomputed-scores) __Done__
      - _Note: Find out how the script of Shuang works on a smaller scale, get an example where a CAPICE score is missing._
  2. [Normalize BM_ratio](#Normalize-BM_ratio) for gene length, so $\frac{n_{malignant}}{n_{benign}}$ per 1000 bases in a gene, instead of [$\frac{n_{malign}}{n_{benign}} \times n_{total}$](output_img/bm_ratio.png "BM_ratio").
  3. _(questionable)_ Fix PCA by identifying categorical features _(Questionable because should I still put time and focus in on the input data?)_
      - _Ask Shuang, Joeri and Krista in the PRU 28-04-2020 @ 13:00-14:00_
  4. [See what is required](#Required-for-data-sources) for the [data sources](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit) to be implemented into the input data.
      - _Note: as this is research, I can use sources that only have precomputed scores available, but just to keep in mind that I should mention that in the discussion._

- Notes:
  - If I find myself unfocussed:
      - Win a game of [solitaire](https://play.google.com/store/apps/details?id=com.lemongame.klondike.solitaire&hl=nl) (game is rigged, not every match is win-able).
      - Clean out my closet (it's messy, maybe to clean up the mess inside my head too).
      - Go for a walk around the neighbourhood.

<br/><br/>
<br/><br/>
### Fixing CAPICE precomputed scores

Problem: CAPICE precomputed file of all single nucleotide level variants contains caps.

Solution: Fix the underlying problem that causes these gaps.

Required: 
- Script to make and batch the CAPICE computing __(positive check)__
- Access to GCC cluster __(positive check)__
- Knowledge of job scheduler on GCC cluster __(positive check)__
- Smaller batches of SNV files to run locally  __(positive check)__

Possible cause:
- Incorrect batching on tabix.

31-04-2020 Problem:
- Got it working on Gearshift, but with a problem:
    - AttributeError: Can't get attribute 'DeprecationDict' on <module 'sklearn.utils.deprecation' from '/apps/software/PythonPlus/3.7.4-foss-2018b-v20.02.1/lib/python3.7/site-packages/sklearn/utils/deprecation.py'>
- Which let's me to beleive that the model was trained in a previous version of sklearn.

<br/><br/>

- Update 10:51 : Model compatibility issue. Might have to retrain CAPICE on Gearshift using the latest xgboost, sklearn, scikit-learn, numpy and pandas just to be safe.

- Update 12:01 : Managed to get it working using python3.6 on calculon, essentially downgrading my own script (I only used python3.7 for F-strings)

- Update 15:55 : Found a very weird bug in the chunking. Seems like the script doesn't like to start at 0 with skiprows and 1000 nrows, solution: nrows - 2 when first iteration, then + 1, then start += nrows + 1...

- Update 04-05-2020 : Found out that the first iteration, an overhead of 2 variants was present. Adjusted the start with 2 to compensate. Seemed fine in the first 400000 CADD entries. (no duplicates nor missed entries). __Started full processing__

- Update 04-05-2020 : 
    - Add functionality to continue where left off if it crashes.
    - Make dummy script to check for duplicates or if the files are fully complete.
    
- Update 06-05-2020 : 
    - Functionaility to continue where left off is required. After 40 hours, it's still processing chromosome 1 after 7 million processed variants. 
        - Ideas: write json file with start and stop each iteration once that part is processed and added to the larger files. Should overwrite.
        - As for now, since this idea is not implemented yet, count the number of rows in the already processed files and set that as start.

- Update 11-05-2020:
    - The 4 days have ended, chromosome 1 has not been finished yet.
    - Output file of chromosome 1 has been renamed to a .tsv file and gzipped (saving about 5 gb of storage).
    - Adjustments to the original script have to be made.
   

#### Version 2.0

- Added gzip output by default
- Currently no function yet to continue where left off.
    - Ideas:
        - for line in gzip.open(file.gz): nlines+=1 (to trace back where was left off)
            - Do not know if this lead to any complications of duplicated entries. Saving the previous results in the RAM memory might solve this to compare to.
    - Then in the future:
        - Save a json of start and batch size each iteration to mark where the file was left of.

- Update 15-05-2020:
    - Program did not load in the previous results correctly, resulting in 12 duplicates (111400001-111400012) (including start and stop). Discuss with Joeri and Shuang what to do. It takes roughly 200 MB of ram to load in 2.1 million processed variants, calculating this for 111.4 million variants (that's how much there is in the file now), would come down to ~11GB of ram usage.

- Update 18-05-2020:
    - Script to remove duplicates from the CAPICE processed file has been run and duplicates have been removed.
    - First test runs have been ran, only for me to realised I messed up. I kept copying the main class, instead of the calculate_capice class. Final release should be on this date.
    

<br/><br/>
<br/><br/>
### Normalize BM_ratio

Problem: Larger genes have more benign and malignant samples, because they are larger and thus the chanses of variation on the reference read is higher.

Solution: Make a $n_{malignant}$ and $n_{benign}$ per $x$ bases in a gene, where {$x \in \mathbb{N}\ | x \geq 1 | x \leq 10000$}

<br/><br/>
<br/><br/>
### Required for data sources

Problem: Now that a lot of data sources have been found, annotation and merging has to be performed in order to use it as training data.
- _Additional problem: once a source has been proven beneficial, it has te be implemented to be annotated before capice fits and predicts a new sample._

Solution: Find out, feature by feature if genes that have a low AUC improve *__and that genes that have a high AUC do not perform worse__*

Required:
- Storage 
    - On GCC cluster for precomputed score files.
    - Locally IF file isn't that big or an API can be used.
- Data sources __([positive check](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit))__
- Script to annotate / merge


      

# CTEPG (Catching The Elusive Predictable Genes) 28-04-2020 - 04-05-2020
<br/><br/>

What have I done:

- [Enrichr Genes < 0.6](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=b7a959357214069d1905719220ed3e3b)
- [Enrichr Genes < 0.7](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=7d252a3e1df275859228fd562e964344)
    - For reference: [Enrichr Genes < 0.85](https://amp.pharm.mssm.edu/Enrichr/enrich?dataset=60c74741c967c25331419bd2b5e7183e)
- [Finished investigating initial data sources](https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit)

<br/><br/>
<br/><br/>

What I'm planning to do this week:

- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores). __Done and not done, script has to have some additonal features__
    - If stuck, do not keep walking into the wall, continue below and contact Shuang and Joeri
- [Run enrichr on increasing cutoffs in AUC (.5, .6, .7 etc.)](#Enrichr-automated-API). __Done__
    - Read into the [Enrichr API documentation](https://amp.pharm.mssm.edu/Enrichr/help#api)
- Mail Tsjerk that a meeting is apreciated. __Done, no response yet__
- Make [suggested plot](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p). __Done__
    - Select from enrichr all the genes involved in (lipid) metabolism and see if this explains all / most of the bad performing genes.
- Make model on bad performing genes and investigate further.
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - Create outline for article.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Fix PCA.
    - Find possible way to annotate genes for co-factors, sub-family etc.
<br/><br/>

# CTEPG (Catching The Elusive Predictable Genes) 04-05-2020 - 11-05-2020
<br/><br/>

What have I done:
- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores).
- [Run enrichr on increasing cutoffs in AUC (.5, .6, .7 etc.)](#Enrichr-automated-API).
- Make [suggested plot](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p).
<br/><br/>

Planned for this week:
- Release precomputed CAPICE script v2.0
- Investigate Bokeh plots on commonalities between Adjusted P-value of 0-0.2 and AUC of 0.5-~0.75 and try to find features / links that would be beneficial to train a separate model on. (or prehaps just train a new model without features and see if performance improves)
    - Ion transport / transmembrane related genes seems to perform bad. 
    - Find commonalities between y-axis 0-0.2 and x-axis 0-0.7
    - Make model on bad performing genes and investigate further.
        - Make plots combined of all sources, but color them by source.
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - Create outline for article.
        - For static plots: make labels on each point with the term name and the number of genes in (n=x) behind that.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python (calculate average AUC for each panel, take the worst performing onces and retain).
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Find possible way to annotate genes for co-factors, sub-family etc.
    - Interactive plot of all variants labeled as benign or pathogenic with their capice score (y-axis) and location (x-axis), to show the variant itself.
<br/><br/>

# CTEPG (Catching The Elusive Predictable Genes) 11-05-2020 - 18-05-2020
<br/><br/>

What have I done:
- [Fixing missing values in precomputed CAPICE scoring file](#Fixing-CAPICE-precomputed-scores).
- Make more bokeh plots on enrichr results
    - Investigate correlations between AUC's and Adjusted P-value.  (spearman / pearson, preverably spearman). Keep in mind that the distribution is not linear, but more like a normal distribution like.
        - Make legend interactive to confirm normal distribution like.
            - To make legend interactive, I have to scatterplot each component separetly, like [this](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/legends.html#hiding-glyphs)
    - Chromosome X seems enriched for higher AUC thresholds. Bias in the source data? (Put in discussion). Lot's of developmental delay reseach is linked / correlated to chromosome X.

Planned for this week:
- __Release precomputed CAPICE script v2.0__
- Run correlation test for all possible sources on Enrichr to maximize chanses of finding stuff.
    - Look into a good way to get all possible results from the enrichr API
- __Escalate mail contact with Tsjerk to Martijn and Fenna.__
- Investigate Bokeh plots on commonalities between Adjusted P-value of 0-0.2 and AUC of 0.5-~0.75 and try to find features / links that would be beneficial to train a separate model on. (or prehaps just train a new model without features and see if performance improves)
    - Ion transport / transmembrane related genes seems to perform bad. 
    - Find commonalities between y-axis 0-0.2 and x-axis 0-0.7
    - __Make model on bad performing genes and investigate further.__
        - Make plots combined of all sources, but color them by source. __Done__
- Start investigating what is required for the data sources to be used in the training phase.
- Look into the genes that do perform well without a CGD category, also look into the 1/2 genes that do not perform well __with__ a CGD category
- _Optionally_ :
    - __Create outline for article.__
        - For static plots: make labels on each point with the term name and the number of genes in (n=x) behind that.
    - Adjust BM_ratio plot to normalize for gene length.
    - Get the UMCG gene panel lists into python (calculate average AUC for each panel, take the worst performing onces and retain).
        - Then: investigate which gene panel is performing the worst.
        - It would be nice to have this on the side for diagnostics, do __not__ focus on this
    - Find possible way to annotate genes for co-factors, sub-family etc.
    - Interactive plot of all variants labeled as benign or pathogenic with their capice score (y-axis) and location (x-axis), to show the variant itself.
<br/><br/>
    
## For after I graduate:
<br/><br/>
Make a script that for every gene, for every gene makes a plot like [these](https://molgenis26.gcc.rug.nl/downloads/capice/plots/), but then interactive that if you hover over a point you see it's chromosomal location, ref and alt and maybe an RS number?
 







  
    

# Current Tickets:
<br/><br/>

## As student / Robert I want to map requirements for SNV and gene features.

- Content:

Overview of features has now been collected and processed (https://docs.google.com/document/d/1D5SiNbeDEfY2hTWquS88MGUrLv6IgZzfCzflppRGb5k/edit). In order to use these features, knowledge has to be obtained for the requirements on said features including, but not limited to merging to the trianing dataset and storage.

- Acceptance criterea:
    - Does not take up terrabytes of storage.
        - _If it does, data sources might have to be merged 1 by 1, removing the source after the merge._
    - Can be merged without 3rd party software (tabix, python etc. allowed).
    - Data can easily be downloaded.

<br/><br/>

## As student / Robert I want to find commonalities between bad performing genes in CAPICE

- Content:

The enrichr tool has proven to be quite beneficial to understanding a set of bad performing genes. Further processing of these enrichr results is required in order to fully understand the underlying mechanics and commonalities that these bad performing genes share.

- Acceptance criterea:
    - Enrichr is performed on multiple gene sets of different AUC thresholds (0.5, 0.6, 0.7 etc.).
    - Results of various data sources (kegg, bioplanet, wikipathways etc.) have been obtained in an automated way using the Enrichr API.
    - Results from the Enrichr API have been processed in an understandable plot, an mockup can be found [here](https://docs.google.com/presentation/d/1_woKoD9nqWnxueyFYfwyX6B4owg8vYF7rw_nKpMPux4/edit#slide=id.p).


# Enrichr automated API

In [4]:
enrichr_url = 'http://amp.pharm.mssm.edu/Enrichr/'

In [5]:
# Load in initial data
data = pd.read_csv(read_loc+'optimal_f1_full_ds_v2.csv', header=0)
data.sort_values(by='default_f1', ascending=False)

,gene,default_auc,default_f1,default_recall,default_fpr,default_spec,optimal_c,optimal_auc,optimal_f1,optimal_recall,optimal_fpr,optimal_spec,n_tot,n_benign,n_malign
2775,PRKN,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,11,7,4
1659,BUB1B,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,73,66,7
2102,GALNT3,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,12,10,2
1655,PNPLA1,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,0.8,0.2,0.8,31,26,5
2782,TMEM80,1.000000,1.0,1.0,0.0,1.0,0.020,1.000000,1.000000,1.0,0.0,1.0,5,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2625,C10orf2,0.469697,0.0,0.0,1.0,0.0,0.020,0.469697,0.000000,0.0,1.0,0.0,133,132,1
2330,SPTLC2,0.455157,0.0,0.0,1.0,0.0,0.011,0.750374,0.170213,0.0,1.0,0.0,229,223,6
2615,PLCD1,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,8,7,1
2611,UBE2A,0.500000,0.0,0.0,1.0,0.0,0.020,0.500000,0.000000,0.0,1.0,0.0,3,2,1


In [6]:
data.shape[0]

2807

In [7]:
# Make a dictionary of lists of genes of interest with as key the 0.x threshold

auc_dict = {}
thresholds = np.arange(0.5, 1.01, 0.05).round(decimals=2)
for t in thresholds:
    list_of_genes = data[data['default_auc'] < t]['gene'].tolist()
    auc_dict[t.astype('str')] = '\n'.join(list_of_genes)
    del list_of_genes
del thresholds

# Save it to a file so I don't bombard the Enrichr API with requests
if not os.path.isfile('./user_id_list.json'):
    file = open('./user_id_list.json', 'w')
    file.close()

with open('./user_id_list.json') as json_file:
    try:
        user_list_ids = json.load(json_file)
        json_was_empty = False
    except json.JSONDecodeError:
        user_list_ids = {}
        json_was_empty = True

In [8]:
# Get list ID's for the different AUC's

for threshold in auc_dict.keys():
    if threshold not in user_list_ids.keys():
        description = f'CAPICE performance of genes with an AUC lower than {threshold}'
        payload = {
            'list': (None, auc_dict[threshold]),
            'description': (None, description)
        }
        response = requests.post(enrichr_url + '/addList', files=payload)
        if not response.ok:
            print(f'Something went wrong with {threshold}!')
        else:
            enrichr_addlist_response = json.loads(response.text)
            user_list_ids[threshold] = enrichr_addlist_response['userListId']

# Write it to a file in case I didn't have any.
if json_was_empty:
    with open('./user_id_list.json', 'w') as json_file:
        json.dump(user_list_ids, json_file)

In [9]:
# Now to get the enrichment results.

enrichr_sources_oi = ['BioPlanet_2019', 'WikiPathways_2019_Human', 'KEGG_2019_Human', 'Elsevier_Pathway_Collection',
                     'GO_Biological_Process_2018','Human_Phenotype_Ontology','Jensen_DISEASES',
                     'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET','OMIM_Disease','OMIM_Expanded',
                     'Human_Gene_Atlas', 'ARCHS4_Tissues',
                     'HMDB_Metabolites','Chromosome_Location_hg19']


In [10]:
# Now get the enrichr results.
def get_enrichr_results(enrichr_sources, number_of_hits):
    if len(enrichr_sources) == 0:
        print('You should give some sources for me to get data from.')
    else:

        # Prepare output dataframes for each source
        output_dir = '/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/'
        
        for iteration, source in enumerate(enrichr_sources):
            print(f'Working on source: {source}')
            if iteration == len(enrichr_sources):
                sources_left = []
            else:
                sources_left = enrichr_sources[iteration+1::]
            print(f'Sources left: {sources_left}')
            # Prepare the output dict
            current_source_df = pd.DataFrame(columns=['AUC',
                                                      'Rank',
                                                      'Term_Name',
                                                      'Overlapping_Genes',
                                                      'Adjusted_P-value',
                                                      'Source'])
            for (threshold, user_id) in zip(user_list_ids.keys(),user_list_ids.values()):
                print(f'\tCurrently working on threshold: {threshold}')
                req = enrichr_url + f'enrich?userListId={user_id}&backgroundType={source}'
                response = requests.get(req)
                if not response.ok:
                    print(f'Userid {user_id} (threshold: {threshold})'
                          f'received error on: {source}!')
                else:
                    enrichr_response_data = json.loads(response.text)
                    content = enrichr_response_data[source]
                    auc = threshold
                    content_df = pd.DataFrame(columns=['AUC', 'Rank', 'Term_Name',
                                                       'Combined_score',
                                                       'Overlapping_Genes',
                                                       'Adjusted_P-value',
                                                       'Source'])
                    for c in content:
                        rank = c[0]
                        term_name = c[1]
                        combined_score = c[4]
                        overlapping_genes = ', '.join(c[5])
                        a_p_value = c[6]
                        local_df = pd.DataFrame({'AUC':auc, 'Rank':rank,
                                                 'Term_Name': term_name,
                                                 'Combined_score': combined_score,
                                                 'Overlapping_Genes': overlapping_genes,
                                                 'Adjusted_P-value': a_p_value,
                                                'Source': source}, index=[0])
                        content_df = content_df.append(local_df, ignore_index=True)
                    content_df = content_df.sort_values(by=['Adjusted_P-value',
                                                            'Combined_score'],
                                                        ascending=[True, False])
                    max_rows = 0
                    if content_df.shape[0] < number_of_hits:
                        max_rows = content_df.shape[0] - 1
                    else:
                        max_rows = number_of_hits
                    out_df = content_df[:max_rows]
                    out_df.drop('Combined_score', axis=1, inplace=True)
                    current_source_df = current_source_df.append(out_df, ignore_index=True)
            output_filename = output_dir + source + '.csv'
            current_source_df.to_csv(output_filename, index=False, sep=',')
            print(f'\tSource {source} has been exported to csv (\n\t\t{output_filename}\n\t)')

In [11]:
output_source_link = {}
for source in enrichr_sources_oi:
    output_filename = source + '.csv'
    output_source_link[source] = output_filename

In [12]:
glob_files_present = glob.glob('./EnrichrAPIResults/*.csv')
list_of_files_present = []
if len(glob_files_present) > 0:
    for file in glob_files_present:
        list_of_files_present.append(file.split('/')[-1])

In [13]:
sources_required = []
for source, output_filename in output_source_link.items():
    if output_filename not in list_of_files_present:
        sources_required.append(source)

In [14]:
sources_required

['BioPlanet_2019',
 'WikiPathways_2019_Human',
 'KEGG_2019_Human',
 'Elsevier_Pathway_Collection',
 'GO_Biological_Process_2018',
 'Human_Phenotype_Ontology',
 'Jensen_DISEASES',
 'PheWeb_2019',
 'ClinVar_2019',
 'GWAS_Catalog_2019',
 'DisGeNET',
 'OMIM_Disease',
 'OMIM_Expanded',
 'Human_Gene_Atlas',
 'ARCHS4_Tissues',
 'HMDB_Metabolites',
 'Chromosome_Location_hg19']

In [15]:
get_enrichr_results(sources_required, 20)

Working on source: BioPlanet_2019
Sources left: ['WikiPathways_2019_Human', 'KEGG_2019_Human', 'Elsevier_Pathway_Collection', 'GO_Biological_Process_2018', 'Human_Phenotype_Ontology', 'Jensen_DISEASES', 'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET', 'OMIM_Disease', 'OMIM_Expanded', 'Human_Gene_Atlas', 'ARCHS4_Tissues', 'HMDB_Metabolites', 'Chromosome_Location_hg19']
	Currently working on threshold: 0.5


/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


	Currently working on threshold: 0.55
	Currently working on threshold: 0.6
	Currently working on threshold: 0.65
	Currently working on threshold: 0.7
	Currently working on threshold: 0.75
	Currently working on threshold: 0.8
	Currently working on threshold: 0.85
	Currently working on threshold: 0.9
	Currently working on threshold: 0.95
	Currently working on threshold: 1.0
	Source BioPlanet_2019 has been exported to csv (
		/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/BioPlanet_2019.csv
	)
Working on source: WikiPathways_2019_Human
Sources left: ['KEGG_2019_Human', 'Elsevier_Pathway_Collection', 'GO_Biological_Process_2018', 'Human_Phenotype_Ontology', 'Jensen_DISEASES', 'PheWeb_2019', 'ClinVar_2019', 'GWAS_Catalog_2019', 'DisGeNET', 'OMIM_Disease', 'OMIM_Expanded', 'Human_Gene_Atlas', 'ARCHS4_Tissues', 'HMDB_Metabolites', 'Chromosome_Location_hg19']
	Currently working on threshold: 0.5
	Currently working on threshold: 0.55
	Currently working on thre

	Currently working on threshold: 0.55
	Currently working on threshold: 0.6
	Currently working on threshold: 0.65
	Currently working on threshold: 0.7
	Currently working on threshold: 0.75
	Currently working on threshold: 0.8
	Currently working on threshold: 0.85
	Currently working on threshold: 0.9
	Currently working on threshold: 0.95
	Currently working on threshold: 1.0
	Source DisGeNET has been exported to csv (
		/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/DisGeNET.csv
	)
Working on source: OMIM_Disease
Sources left: ['OMIM_Expanded', 'Human_Gene_Atlas', 'ARCHS4_Tissues', 'HMDB_Metabolites', 'Chromosome_Location_hg19']
	Currently working on threshold: 0.5
	Currently working on threshold: 0.55
	Currently working on threshold: 0.6
	Currently working on threshold: 0.65
	Currently working on threshold: 0.7
	Currently working on threshold: 0.75
	Currently working on threshold: 0.8
	Currently working on threshold: 0.85
	Currently working on threshold

In [16]:
enrichr_data_loc = './EnrichrAPIResults/'

enrichr_dataset = {}

for source in enrichr_sources_oi:
    dataset = enrichr_data_loc + source + '.csv'
    enrichr_dataset[source] = pd.read_csv(dataset)

In [17]:
enrichr_dataset['BioPlanet_2019'].head()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019


In [18]:
def plot_results(source, n=10, print_messages=False, single_plot = False):
    source_name = source['Source'][0]
    if not os.path.exists(f'./EnrichrAPIResults/Plots_for_{n}_categories'):
        os.mkdir(f'./EnrichrAPIResults/Plots_for_{n}_categories')
    output_file(filename=f'./EnrichrAPIResults/Plots_for_{n}_categories/n_{n}_results_from_{source_name}.html',
               title=f'Bokeh plot of {source_name} (n={n})')
    term_values = source['Term_Name'].value_counts()
    if print_messages:
        print(f'The most occuring term names in {source_name} is: \n{term_values[:n]}')
    plot_source = pd.DataFrame(columns=source.columns)
    for auc in source['AUC'].unique():
        subsource = source[source['AUC'] == auc][:n]
        plot_source = plot_source.append(subsource)
    category = 'Term_Name'
    category_items = plot_source[category].unique()
    palette = inferno(len(category_items) + 1)
    colormap = dict(zip(category_items, palette))
    plot_source['color'] = plot_source[category].map(colormap)
    
    title = f'Term names from {source_name}, n={n}'
    source_bokeh = ColumnDataSource(plot_source)
    hover = HoverTool(tooltips=[('Term_name', '@Term_Name'), ('Genes', '@Overlapping_Genes')])
    tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), SaveTool()]
    
    p = figure(tools=tools, title=title, plot_width=2000, plot_height=1200,toolbar_location='right', toolbar_sticky=False,)
    if n > 5:
        p.scatter(x='AUC',y='Adjusted_P-value', source=source_bokeh, size=10, color='color')
    else:
        p.scatter(x='AUC',y='Adjusted_P-value', source=source_bokeh, size=10, color='color', legend=category)
    p.xaxis.axis_label = 'AUC'
    p.yaxis.axis_label = 'Adjusted P-value'
    return p
    

In [19]:
for key in enrichr_dataset.keys():
    plot = plot_results(enrichr_dataset[key], 10)
    save(plot)

In [20]:
combined_df = pd.DataFrame(columns=enrichr_dataset['BioPlanet_2019'].columns)
for key, value in enrichr_dataset.items():
    subdf = pd.DataFrame(columns=combined_df.columns)
    for auc in value['AUC'].unique():
        subdf = subdf.append(value[value['AUC'] == auc][:10], ignore_index=True)
    combined_df = combined_df.append(subdf, ignore_index=True)
combined_df

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
...,...,...,...,...,...,...
1864,1.0,6,chr21,"APP, COL18A1, KCNE1, KCNE2, AIRE, TMPRSS3, ITG...",1.00000,Chromosome_Location_hg19
1865,1.0,7,chr14,"POMT2, ABCD4, SERPINA1, CHD8, PYGL, GPHN, IRF2...",1.00000,Chromosome_Location_hg19
1866,1.0,8,chr22,"SMARCB1, NCF4, MLC1, TCF20, TRIOBP, PEX26, CYP...",1.00000,Chromosome_Location_hg19
1867,1.0,9,chr6 ssto hap7,"SYNGAP1, TNXB, CYP21A2, TUBB, CSNK2B, NEU1, CO...",1.00000,Chromosome_Location_hg19


In [21]:
combined_df.head(20)

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.50,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.50,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.50,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.50,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.50,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
5,0.50,15,Elevation of cytosolic calcium levels,TRPC6,1.00000,BioPlanet_2019
6,0.50,18,Second messenger role in netrin-1 signaling,TRPC6,1.00000,BioPlanet_2019
7,0.50,20,Signaling by activated point mutants of FGFR1,FGF9,1.00000,BioPlanet_2019
8,0.50,21,Caspase-mediated cleavage of cytoskeletal prot...,SPTAN1,1.00000,BioPlanet_2019
9,0.50,22,Tandem pore domain potassium channels,KCNK9,1.00000,BioPlanet_2019


In [69]:
def plot_results_in_singleplot(source):
    output_file(filename='./EnrichrAPIResults/combined_enrichr_plot_results.html',
               title='Combined enrichr data sources plot')
    category = 'Source'
    category_items = source[category].unique()
    palette = inferno(len(category_items) + 1)
    colormap = dict(zip(category_items, palette))
    print(colormap)
    source['color'] = source[category].map(colormap)
    
    title = f'Combined enrichr plot results.'
    source_bokeh = ColumnDataSource(source)
    hover = HoverTool(tooltips=[('Term_name', '@Term_Name'),
                                ('Genes', '@Overlapping_Genes'),
                               ('Source', '@Source')])
    tools = [hover, WheelZoomTool(), PanTool(), BoxZoomTool(), ResetTool(), SaveTool()]
    
    p = figure(tools=tools, title=title, plot_width=2000, plot_height=1200,toolbar_location='right', toolbar_sticky=False,)
    p.scatter(x='AUC',y='Adjusted_P-value', source=source_bokeh, size=10, color='color', legend=category)
    p.xaxis.axis_label = 'AUC'
    p.yaxis.axis_label = 'Adjusted P-value'
    p.legend.click_policy="hide"
    return p

In [ ]:
plot = plot_results_in_singleplot(combined_df)
save(plot)

In [ ]:
combined_df_ext = pd.DataFrame(columns=enrichr_dataset['BioPlanet_2019'].columns)
for key, value in enrichr_dataset.items():
    subdf = pd.DataFrame(columns=combined_df_ext.columns)
    for auc in value['AUC'].unique():
        subdf = subdf.append(value[value['AUC'] == auc], ignore_index=True)
    combined_df_ext = combined_df_ext.append(subdf, ignore_index=True)
combined_df_ext

In [ ]:
combined_df_ext.where((combined_df_ext['AUC'] == 0.5) & (combined_df_ext['Source'] == 'BioPlanet_2019')).dropna()

In [ ]:
# https://gist.github.com/deekayen/4148741#file-1-1000-txt

common_used_words = []
with open('/home/rjsietsma/Documents/1-1000.txt') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        common_used_words.append(line)

In [ ]:
word_count_df = pd.DataFrame(columns=['word', 'count', 'auc'])
for auc in combined_df_ext['AUC'].unique():
    word_list = []
    for source in combined_df_ext['Source'].unique():
        subset = combined_df_ext.where((combined_df_ext['AUC'] == auc) & (combined_df_ext['Source'] == source)).dropna()
        for i, row in subset.iterrows():
            term_name = row['Term_Name']
            word_list += term_name.lower().split()
    word_count_dict = {}
    for word in word_list:
        if word in common_used_words:
            continue
        if word not in word_count_dict.keys():
            word_count_dict[word] = 1
        else:
            word_count_dict[word] += 1
    temp_df = pd.DataFrame(columns=word_count_df.columns)
    for key in word_count_dict.keys():
        temp_temp_df = pd.DataFrame({'word':key, 'count': word_count_dict[key],
                                    'auc': auc}, index=[0])
        temp_df = temp_df.append(temp_temp_df, ignore_index=True)
    word_count_df = word_count_df.append(temp_df, ignore_index=True)

In [ ]:
word_count_df.sort_values(by=['auc','count'], ascending=[True, False], inplace=True)

In [ ]:
nwkdf = pd.DataFrame(columns=word_count_df.columns)
for auc in word_count_df['auc'].unique():
    nwkdf = nwkdf.append(word_count_df[word_count_df['auc'] == auc][:5])
nwkdf

In [ ]:
def plot_word_count_results(source):
    output_file(filename='./EnrichrAPIResults/enrichr_word_count_bokeh.html',
                title='Word count plot')
    category = 'word'
    category_items = source[category].unique()
    palette = inferno(len(category_items) + 1)
    colormap = dict(zip(category_items, palette))
    source['color'] = source[category].map(colormap)
    
    source['auc'] = source['auc'].astype(str)
    
    group = source.groupby(by=['auc', 'word']).sum()
    group.sort_values(by=['auc','count'], ascending=[True,False], inplace=True)
    
    title = f'Word count bar plot'
    
    factors = group.index.tolist()
    x = group['count'].tolist()
    x_str = group['count'].astype(str).tolist()
    colors = group['color'].tolist()
    
    temp_source = ColumnDataSource(source)
        
    tooltips = [('Counts', '@count')]
    
    p = figure(x_range=FactorRange(*factors),
               title=title,
               plot_width=2000,
               plot_height=1200,
               toolbar_location='right',
               toolbar_sticky=False)
    p.vbar(x=factors,
           top=x,
           color=colors,
           width=0.9)
    p.add_tools(HoverTool(tooltips=[('Counts','@top')]))
    p.xaxis.axis_label = 'Word'
    p.yaxis.axis_label = 'Count'
    p.xaxis.major_label_orientation = 1.5
    return p

In [ ]:
plot = plot_word_count_results(nwkdf)
save(plot)

In [ ]:
gene_count_df = pd.DataFrame(columns=['gene', 'count', 'auc'])
for auc in combined_df_ext['AUC'].unique():
    subset_df = combined_df_ext[combined_df_ext['AUC'] == auc]
    gene_list = []
    unsplitted_gene_list = subset_df['Overlapping_Genes'].tolist()
    for unsplitted_gene in unsplitted_gene_list:
        gene_list += unsplitted_gene.split(', ')
    gene_count_dict = {}
    for gene in gene_list:
        if gene not in gene_count_dict.keys():
            gene_count_dict[gene] = 1
        else:
            gene_count_dict[gene] += 1
    temp_df = pd.DataFrame(columns=gene_count_df.columns)
    for key in gene_count_dict.keys():
        temp_temp_df = pd.DataFrame({'gene':key, 'count': gene_count_dict[key],
                                    'auc': auc}, index=[0])
        temp_df = temp_df.append(temp_temp_df, ignore_index=True)
    gene_count_df = gene_count_df.append(temp_df, ignore_index=True)

In [ ]:
gene_count_df.sort_values(by=['auc','count'], ascending=[True, False], inplace=True)

In [ ]:
ngkdf = pd.DataFrame(columns=gene_count_df.columns)
for auc in gene_count_df['auc'].unique():
    ngkdf = ngkdf.append(gene_count_df[gene_count_df['auc'] == auc][:5])

In [ ]:
def plot_gene_count_results(source):
    output_file(filename='./EnrichrAPIResults/enrichr_gene_count_bokeh.html',
                title='Gene count plot')
    
    category = 'gene'
    category_items = source[category].unique()
    palette = inferno(len(category_items) + 1)
    colormap = dict(zip(category_items, palette))
    source['color'] = source[category].map(colormap)
    
    source['auc'] = source['auc'].astype(str)
    
    group = source.groupby(by=['auc', 'gene']).sum()
    group.sort_values(by=['auc','count'], ascending=[True,False], inplace=True)
    
    title = f'Gene count bar plot'
    
    factors = group.index.tolist()
    x = group['count'].tolist()
    x_str = group['count'].astype(str).tolist()
    colors = group['color'].tolist()
    
    temp_source = ColumnDataSource(source)
        
    tooltips = [('Counts', '@count')]
    
    p = figure(x_range=FactorRange(*factors),
               title=title,
               plot_width=2000,
               plot_height=1200,
               toolbar_location='right',
               toolbar_sticky=False)
    p.vbar(x=factors,
           top=x,
           color=colors,
           width=0.9)
    p.add_tools(HoverTool(tooltips=[('Counts','@top')]))
    p.xaxis.axis_label = 'Gene'
    p.yaxis.axis_label = 'Count'
    p.xaxis.major_label_orientation = 1.5
    return p

In [ ]:
plot = plot_gene_count_results(ngkdf)
save(plot)

In [ ]:
cdfe = combined_df_ext.copy()
cdfed = cdfe.copy()
cdfe.drop(['Overlapping_Genes', 'Rank'], axis=1, inplace=True)

In [ ]:
fcdfe = pd.DataFrame(columns=cdfe.columns)
for auc in cdfe['AUC'].unique():
    subset = cdfe[cdfe['AUC'] == auc]
    subset.sort_values(by='Adjusted_P-value', ascending=True, inplace=True)
    subset = subset[:5]
    fcdfe = fcdfe.append(subset, ignore_index=True)
fcdfe.to_csv('./EnrichrAPIResults/combined_df_nondetailed_paper.csv', index=False)
fcdfe

In [ ]:
fcdfed = pd.DataFrame(columns=cdfed.columns)
for auc in cdfed['AUC'].unique():
    subset = cdfed[cdfed['AUC'] == auc]
    subset.sort_values(by='Adjusted_P-value', ascending=True, inplace=True)
    subset = subset[:5]
    fcdfed = fcdfed.append(subset, ignore_index=True)
fcdfed.to_csv('./EnrichrAPIResults/combined_df_detailed_paper.csv', index=False)
fcdfed

In [ ]:
fcdfede = pd.DataFrame(columns=cdfed.columns)
for auc in cdfed['AUC'].unique():
    subset = cdfed[cdfed['AUC'] == auc]
    subset.sort_values(by='Adjusted_P-value', ascending=True, inplace=True)
    fcdfede = fcdfede.append(subset, ignore_index=True)
fcdfede.to_csv('./EnrichrAPIResults/combined_df_detailed_paper_extended.csv', index=False)
fcdfede

## Interpretation
It seems that a normal distribution like pattern is formed with a steeper slope after the maximum, starting at rougly AUC of 0.3 and ending at roughly 0.8.
<br/><br/>
- DisGeNET seems to outline an interesting picture with the top 10 term names:
    - For it seems that in the really bad performing genes the main cause could be infant development related. Later on, more neurological causes are identified.
        - Neurological seems clear to me... There is still so much unknown about the human mind.
        - Infant development related is a bit less clear, since there is a growing amount of research towards this infant development.
- GO_biological_process seems to have primarily the same genes for AUC 0.55-0.7 that are most enriched
    - It seems linked to ion channeling
- Feet and toe abnormalities seem to occur once or twice too
- Could this be an indication to genes that are related to very small molecules? (like steroids, ions, neuro-molecules?)
- Most significant: autosomal dominant inheritence
    - Very interesting plot, the HPO bokeh plot of n=10
- 

<br/><br/>
TODO: make word counts of genes and term names?


In [70]:
## Interpretation
It seems that a normal distribution like pattern is formed with a steeper slope after the maximum, starting at rougly AUC of 0.3 and ending at roughly 0.8.
<br/><br/>
- DisGeNET seems to outline an interesting picture with the top 10 term names:
    - For it seems that in the really bad performing genes the main cause could be infant development related. Later on, more neurological causes are identified.
        - Neurological seems clear to me... There is still so much unknown about the human mind.
        - Infant development related is a bit less clear, since there is a growing amount of research towards this infant development.
- GO_biological_process seems to have primarily the same genes for AUC 0.55-0.7 that are most enriched
    - It seems linked to ion channeling
- Feet and toe abnormalities seem to occur once or twice too
- Could this be an indication to genes that are related to very small molecules? (like steroids, ions, neuro-molecules?)
- Most significant: autosomal dominant inheritence
    - Very interesting plot, the HPO bokeh plot of n=10
- 

<br/><br/>
TODO: make word counts of genes and term names?


{'BioPlanet_2019': '#000003', 'WikiPathways_2019_Human': '#090621', 'KEGG_2019_Human': '#1D0C45', 'Elsevier_Pathway_Collection': '#370961', 'GO_Biological_Process_2018': '#500D6C', 'Human_Phenotype_Ontology': '#68166E', 'Jensen_DISEASES': '#801F6B', 'PheWeb_2019': '#982765', 'ClinVar_2019': '#AF315B', 'GWAS_Catalog_2019': '#C53D4D', 'DisGeNET': '#D94D3D', 'OMIM_Disease': '#E8612B', 'OMIM_Expanded': '#F37918', 'Human_Gene_Atlas': '#FA9306', 'ARCHS4_Tissues': '#FBB014', 'HMDB_Metabolites': '#F8CD37', 'Chromosome_Location_hg19': '#F1E968'}


'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/combined_enrichr_plot_results.html'

In [24]:
combined_df_ext = pd.DataFrame(columns=enrichr_dataset['BioPlanet_2019'].columns)
for key, value in enrichr_dataset.items():
    subdf = pd.DataFrame(columns=combined_df_ext.columns)
    for auc in value['AUC'].unique():
        subdf = subdf.append(value[value['AUC'] == auc], ignore_index=True)
    combined_df_ext = combined_df_ext.append(subdf, ignore_index=True)
combined_df_ext

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
...,...,...,...,...,...,...
3698,1.0,15,chr6 dbb hap3,"TNXB, CYP21A2, TUBB, CSNK2B, NEU1, COL11A2, SK...",1.00000,Chromosome_Location_hg19
3699,1.0,19,chr20,"PTPRT, SNAP25, PANK2, PIGT, MOCS3, ABHD12, JPH...",1.00000,Chromosome_Location_hg19
3700,1.0,17,chr6 qbl hap6,"TNXB, CYP21A2, TUBB, CSNK2B, NEU1, COL11A2, SK...",1.00000,Chromosome_Location_hg19
3701,1.0,20,chr11,"CARS, DPAGT1, CLPB, CTNND1, DNHD1, SBF2-AS1, B...",1.00000,Chromosome_Location_hg19


In [25]:
combined_df_ext.where((combined_df_ext['AUC'] == 0.5) & (combined_df_ext['Source'] == 'BioPlanet_2019')).dropna()

,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Synaptic proteins at the synaptic junction,"NRXN3, SPTAN1",0.90841,BioPlanet_2019
1,0.5,10,Activation of TRKA receptors,NTRK2,1.00000,BioPlanet_2019
2,0.5,12,FGFR3b ligand binding and activation,FGF9,1.00000,BioPlanet_2019
3,0.5,13,Folate and pterine metabolism,SLC46A1,1.00000,BioPlanet_2019
4,0.5,14,Tat-mediated elongation of the HIV-1 transcript,CTDP1,1.00000,BioPlanet_2019
5,0.5,15,Elevation of cytosolic calcium levels,TRPC6,1.00000,BioPlanet_2019
6,0.5,18,Second messenger role in netrin-1 signaling,TRPC6,1.00000,BioPlanet_2019
7,0.5,20,Signaling by activated point mutants of FGFR1,FGF9,1.00000,BioPlanet_2019
8,0.5,21,Caspase-mediated cleavage of cytoskeletal prot...,SPTAN1,1.00000,BioPlanet_2019
9,0.5,22,Tandem pore domain potassium channels,KCNK9,1.00000,BioPlanet_2019


In [26]:
# https://gist.github.com/deekayen/4148741#file-1-1000-txt

common_used_words = []
with open('/home/rjsietsma/Documents/1-1000.txt') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip()
        common_used_words.append(line)

In [27]:
word_count_df = pd.DataFrame(columns=['word', 'count', 'auc'])
for auc in combined_df_ext['AUC'].unique():
    word_list = []
    for source in combined_df_ext['Source'].unique():
        subset = combined_df_ext.where((combined_df_ext['AUC'] == auc) & (combined_df_ext['Source'] == source)).dropna()
        for i, row in subset.iterrows():
            term_name = row['Term_Name']
            word_list += term_name.lower().split()
    word_count_dict = {}
    for word in word_list:
        if word in common_used_words:
            continue
        if word not in word_count_dict.keys():
            word_count_dict[word] = 1
        else:
            word_count_dict[word] += 1
    temp_df = pd.DataFrame(columns=word_count_df.columns)
    for key in word_count_dict.keys():
        temp_temp_df = pd.DataFrame({'word':key, 'count': word_count_dict[key],
                                    'auc': auc}, index=[0])
        temp_df = temp_df.append(temp_temp_df, ignore_index=True)
    word_count_df = word_count_df.append(temp_df, ignore_index=True)

In [28]:
word_count_df.sort_values(by=['auc','count'], ascending=[True, False], inplace=True)

In [68]:
nwkdf = pd.DataFrame(columns=word_count_df.columns)
for auc in word_count_df['auc'].unique():
    nwkdf = nwkdf.append(word_count_df[word_count_df['auc'] == auc][:5])
nwkdf

,word,count,auc
23,signaling,14,0.50
52,pathway,14,0.50
42,regulation,12,0.50
85,disease,12,0.50
317,syndrome,9,0.50
592,signaling,16,0.55
628,pathway,13,0.55
691,disease,12,0.55
761,syndrome,11,0.55
600,metabolism,10,0.55


In [66]:
def plot_word_count_results(source):
    output_file(filename='./EnrichrAPIResults/enrichr_word_count_bokeh.html',
                title='Word count plot')
    category = 'word'
    category_items = source[category].unique()
    palette = inferno(len(category_items) + 1)
    colormap = dict(zip(category_items, palette))
    source['color'] = source[category].map(colormap)
    
    source['auc'] = source['auc'].astype(str)
    
    group = source.groupby(by=['auc', 'word']).sum()
    group.sort_values(by=['auc','count'], ascending=[True,False], inplace=True)
    
    title = f'Word count bar plot'
    
    factors = group.index.tolist()
    x = group['count'].tolist()
    x_str = group['count'].astype(str).tolist()
    colors = group['color'].tolist()
    
    temp_source = ColumnDataSource(source)
        
    tooltips = [('Counts', '@count')]
    
    p = figure(x_range=FactorRange(*factors),
               title=title,
               plot_width=2000,
               plot_height=1200,
               toolbar_location='right',
               toolbar_sticky=False)
    p.vbar(x=factors,
           top=x,
           color=colors,
           width=0.9)
    p.add_tools(HoverTool(tooltips=[('Counts','@top')]))
    p.xaxis.axis_label = 'Word'
    p.yaxis.axis_label = 'Count'
    p.xaxis.major_label_orientation = 1.5
    return p

In [67]:
plot = plot_word_count_results(nwkdf)
save(plot)

/home/rjsietsma/PycharmProjects/dsls_master_thesis/venv/lib/python3.8/site-packages/bokeh/models/plots.py:764: UserWarning: 
You are attempting to set `plot.legend.click_policy` on a plot that has zero legends added, this will have no effect.

Before legend properties can be set, you must add a Legend explicitly, or call a glyph method with a legend parameter set.

  warnings.warn(_LEGEND_EMPTY_WARNING % attr)


'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_word_count_bokeh.html'

In [32]:
gene_count_df = pd.DataFrame(columns=['gene', 'count', 'auc'])
for auc in combined_df_ext['AUC'].unique():
    subset_df = combined_df_ext[combined_df_ext['AUC'] == auc]
    gene_list = []
    unsplitted_gene_list = subset_df['Overlapping_Genes'].tolist()
    for unsplitted_gene in unsplitted_gene_list:
        gene_list += unsplitted_gene.split(', ')
    gene_count_dict = {}
    for gene in gene_list:
        if gene not in gene_count_dict.keys():
            gene_count_dict[gene] = 1
        else:
            gene_count_dict[gene] += 1
    temp_df = pd.DataFrame(columns=gene_count_df.columns)
    for key in gene_count_dict.keys():
        temp_temp_df = pd.DataFrame({'gene':key, 'count': gene_count_dict[key],
                                    'auc': auc}, index=[0])
        temp_df = temp_df.append(temp_temp_df, ignore_index=True)
    gene_count_df = gene_count_df.append(temp_df, ignore_index=True)

In [33]:
gene_count_df.sort_values(by=['auc','count'], ascending=[True, False], inplace=True)

In [34]:
ngkdf = pd.DataFrame(columns=gene_count_df.columns)
for auc in gene_count_df['auc'].unique():
    ngkdf = ngkdf.append(gene_count_df[gene_count_df['auc'] == auc][:5])

In [35]:
def plot_gene_count_results(source):
    output_file(filename='./EnrichrAPIResults/enrichr_gene_count_bokeh.html',
                title='Gene count plot')
    
    category = 'gene'
    category_items = source[category].unique()
    palette = inferno(len(category_items) + 1)
    colormap = dict(zip(category_items, palette))
    source['color'] = source[category].map(colormap)
    
    source['auc'] = source['auc'].astype(str)
    
    group = source.groupby(by=['auc', 'gene']).sum()
    group.sort_values(by=['auc','count'], ascending=[True,False], inplace=True)
    
    title = f'Gene count bar plot'
    
    factors = group.index.tolist()
    x = group['count'].tolist()
    x_str = group['count'].astype(str).tolist()
    colors = group['color'].tolist()
    
    temp_source = ColumnDataSource(source)
        
    tooltips = [('Counts', '@count')]
    
    p = figure(x_range=FactorRange(*factors),
               title=title,
               plot_width=2000,
               plot_height=1200,
               toolbar_location='right',
               toolbar_sticky=False)
    p.vbar(x=factors,
           top=x,
           color=colors,
           width=0.9)
    p.add_tools(HoverTool(tooltips=[('Counts','@top')]))
    p.xaxis.axis_label = 'Gene'
    p.yaxis.axis_label = 'Count'
    p.xaxis.major_label_orientation = 1.5
    return p

In [36]:
plot = plot_gene_count_results(ngkdf)
save(plot)

'/home/rjsietsma/PycharmProjects/dsls_master_thesis/side_scripts/EnrichrAPIResults/enrichr_gene_count_bokeh.html'

In [41]:
cdfe = combined_df_ext.copy()
cdfed = cdfe.copy()
cdfe.drop(['Overlapping_Genes', 'Rank'], axis=1, inplace=True)

In [54]:
fcdfe = pd.DataFrame(columns=cdfe.columns)
for auc in cdfe['AUC'].unique():
    subset = cdfe[cdfe['AUC'] == auc]
    subset.sort_values(by='Adjusted_P-value', ascending=True, inplace=True)
    subset = subset[:5]
    fcdfe = fcdfe.append(subset, ignore_index=True)
fcdfe.to_csv('./EnrichrAPIResults/combined_df_nondetailed_paper.csv', index=False)
fcdfe

<ipython-input-54-cf2ab7324a6e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset.sort_values(by='Adjusted_P-value', ascending=True, inplace=True)


,AUC,Term_Name,Adjusted_P-value,Source
0,0.50,Autosomal dominant inheritance (HP:0000006),3.156383e-12,Human_Phenotype_Ontology
1,0.50,Intellectual Disability,4.370632e-04,DisGeNET
2,0.50,Short stature,1.808020e-03,DisGeNET
3,0.50,Neurodegenerative disease,5.007898e-03,Jensen_DISEASES
4,0.50,neuropathy,5.197129e-03,OMIM_Disease
5,0.55,Autosomal dominant inheritance (HP:0000006),8.394258e-11,Human_Phenotype_Ontology
6,0.55,Autosomal recessive inheritance (HP:0000007),3.454317e-04,Human_Phenotype_Ontology
7,0.55,Neurodegenerative disease,1.440183e-03,Jensen_DISEASES
8,0.55,Intellectual Disability,2.308717e-03,DisGeNET
9,0.55,Abnormal EKG (HP:0003115),6.518371e-03,Human_Phenotype_Ontology


In [55]:
fcdfed = pd.DataFrame(columns=cdfed.columns)
for auc in cdfed['AUC'].unique():
    subset = cdfed[cdfed['AUC'] == auc]
    subset.sort_values(by='Adjusted_P-value', ascending=True, inplace=True)
    subset = subset[:5]
    fcdfed = fcdfed.append(subset, ignore_index=True)
fcdfed.to_csv('./EnrichrAPIResults/combined_df_detailed_paper.csv', index=False)
fcdfed

<ipython-input-55-81717ed15518>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset.sort_values(by='Adjusted_P-value', ascending=True, inplace=True)


,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.50,1,Autosomal dominant inheritance (HP:0000006),"YARS, RTN2, KCNJ5, NTRK2, CEACAM16, KCNK9, MAP...",3.156383e-12,Human_Phenotype_Ontology
1,0.50,1,Intellectual Disability,"RTN2, CD96, SLC46A1, UNC13A, CTDP1, KCNK9, TGD...",4.370632e-04,DisGeNET
2,0.50,2,Short stature,"CD96, DPP6, CTDP1, TGDS, GJB3, GPD1, MYH8, NRX...",1.808020e-03,DisGeNET
3,0.50,1,Neurodegenerative disease,"YARS, RTN2, DPP6, C10ORF2, ANG, TUBA4A, GIGYF2",5.007898e-03,Jensen_DISEASES
4,0.50,1,neuropathy,"CTDP1, GJB3, C10ORF2",5.197129e-03,OMIM_Disease
5,0.55,1,Autosomal dominant inheritance (HP:0000006),"RTN2, KCNK9, KCNE3, KRT86, GIGYF2, KBTBD13, LM...",8.394258e-11,Human_Phenotype_Ontology
6,0.55,2,Autosomal recessive inheritance (HP:0000007),"CD96, SLC25A3, SLC46A1, CTDP1, TPMT, PRCD, NDU...",3.454317e-04,Human_Phenotype_Ontology
7,0.55,1,Neurodegenerative disease,"RTN2, YARS, DPP6, GRID2, C10ORF2, ANG, SMN1, P...",1.440183e-03,Jensen_DISEASES
8,0.55,1,Intellectual Disability,"RTN2, SLC46A1, CTDP1, KCNK9, PRCD, NRXN3, AFF4...",2.308717e-03,DisGeNET
9,0.55,3,Abnormal EKG (HP:0003115),"KCNJ5, CACNB2, KCNE3, TRPM4",6.518371e-03,Human_Phenotype_Ontology


In [56]:
fcdfede = pd.DataFrame(columns=cdfed.columns)
for auc in cdfed['AUC'].unique():
    subset = cdfed[cdfed['AUC'] == auc]
    subset.sort_values(by='Adjusted_P-value', ascending=True, inplace=True)
    fcdfede = fcdfede.append(subset, ignore_index=True)
fcdfede.to_csv('./EnrichrAPIResults/combined_df_detailed_paper_extended.csv', index=False)
fcdfede

<ipython-input-56-6fd3d5e1d73c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset.sort_values(by='Adjusted_P-value', ascending=True, inplace=True)


,AUC,Rank,Term_Name,Overlapping_Genes,Adjusted_P-value,Source
0,0.5,1,Autosomal dominant inheritance (HP:0000006),"YARS, RTN2, KCNJ5, NTRK2, CEACAM16, KCNK9, MAP...",3.156383e-12,Human_Phenotype_Ontology
1,0.5,1,Intellectual Disability,"RTN2, CD96, SLC46A1, UNC13A, CTDP1, KCNK9, TGD...",4.370632e-04,DisGeNET
2,0.5,2,Short stature,"CD96, DPP6, CTDP1, TGDS, GJB3, GPD1, MYH8, NRX...",1.808020e-03,DisGeNET
3,0.5,1,Neurodegenerative disease,"YARS, RTN2, DPP6, C10ORF2, ANG, TUBA4A, GIGYF2",5.007898e-03,Jensen_DISEASES
4,0.5,1,neuropathy,"CTDP1, GJB3, C10ORF2",5.197129e-03,OMIM_Disease
...,...,...,...,...,...,...
3698,1.0,4,chr16,"ROGDI, XYLT1, GCSH, CDH3, SALL1, CDH1, TUBB3, ...",1.000000e+00,Chromosome_Location_hg19
3699,1.0,3,chr18,"CTDP1, FECH, PIGN, LAMA1, TSHZ1, LAMA3, RAX, G...",1.000000e+00,Chromosome_Location_hg19
3700,1.0,2,chr17 ctg5 hap1,"KANSL1, MAPT",1.000000e+00,Chromosome_Location_hg19
3701,1.0,10,chr9,"INVS, LRSAM1, PIGO, LAMC3, GLDC, DOCK8, POMT1,...",1.000000e+00,Chromosome_Location_hg19


## Interpretation
It seems that a normal distribution like pattern is formed with a steeper slope after the maximum, starting at rougly AUC of 0.3 and ending at roughly 0.8.
<br/><br/>
- DisGeNET seems to outline an interesting picture with the top 10 term names:
    - For it seems that in the really bad performing genes the main cause could be infant development related. Later on, more neurological causes are identified.
        - Neurological seems clear to me... There is still so much unknown about the human mind.
        - Infant development related is a bit less clear, since there is a growing amount of research towards this infant development.
- GO_biological_process seems to have primarily the same genes for AUC 0.55-0.7 that are most enriched
    - It seems linked to ion channeling
- Feet and toe abnormalities seem to occur once or twice too
- Could this be an indication to genes that are related to very small molecules? (like steroids, ions, neuro-molecules?)
- Most significant: autosomal dominant inheritence
    - Very interesting plot, the HPO bokeh plot of n=10
- 

<br/><br/>
TODO: make word counts of genes and term names?
